In [1]:
#Import required libraries and read data
import pandas as pd
import numpy as np
import csv
import random
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import ensemble
from sklearn.metrics import log_loss

print('Load data...')
train = pd.read_csv("../train.csv")
#print(train.head())
target = train['target'].values
test = pd.read_csv("../test.csv")
id_test = test['ID'].values
id_train = train['ID'].values
#print(test.head())
train = train.drop(['ID','target','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
test = test.drop(['ID','v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
#print(train.columns.values)
#print(test.columns.values)
#print(train.dtypes)
#print(test.dtypes)

Load data...


In [2]:
#This part of code converts all factor variables into integers and adds -999 to NAs. I tried it to replicate this in R.
print('Clearing...')
for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        #for objects: factorize
        train[train_name], tmp_indexer = pd.factorize(train[train_name])
        test[test_name] = tmp_indexer.get_indexer(test[test_name])
        #but now we have -1 values (NaN)
    else:
        #for int or float: fill NaN
        tmp_len = len(train[train_series.isnull()])
        if tmp_len>0:
            #print "mean", train_series.mean()
            train.loc[train_series.isnull(), train_name] = -999 
        #and Test
        tmp_len = len(test[test_series.isnull()])
        if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = -999

#print(train.columns.values)
#print(test.columns.values)
#print(train.dtypes)
#print(test.dtypes)

Clearing...


In [4]:
#10 fold training and prediction.
NFOLD = float(10)
for i in range(1,(int(NFOLD)+1)):
    st = math.ceil(len(train.axes[0])/NFOLD)
    start = int((st*(i-1)) + 1)
    end = int(min((st*i),len(train.axes[0])))
    train_validation = train[start-1:end]
    target_validation = target[start-1:end]
    train_train = train.drop(train.index[start-1:end])
    target_train = np.delete(target,range(start-1,end),axis=0)
    random.seed(123)
    X_train = train_train
    X_val = train_validation
    X_test = test
    print('Training ',i,'of',NFOLD)
    extc = ExtraTreesClassifier(n_estimators=850,max_features= 60,criterion= 'entropy',min_samples_split= 4,
                            max_depth= 40, min_samples_leaf= 2, n_jobs = -1)
    extc.fit(X_train,target_train)
    
    if i==1:
        train_pred = extc.predict_proba(X_val)
        print("CV Score ",log_loss(target_validation,train_pred))
        train_pred = train_pred[:,1]
        test_pred = extc.predict_proba(X_test)
        test_pred = test_pred[:,1]
    else:
        trp = extc.predict_proba(X_val)
        print("CV Score ",log_loss(target_validation,trp))
        trp = trp[:,1]
        tsp = extc.predict_proba(X_test)
        tsp = tsp[:,1]
        train_pred = np.concatenate((train_pred,trp),axis=0)
        test_pred = test_pred+tsp

print('Writing output files')
test_pred = test_pred/NFOLD
pd.DataFrame({"ID": id_test, "PredictedProb": test_pred}).to_csv('test_predictions_etc_v1.csv',index=False)
pd.DataFrame({"ID": id_train, "PredictedProb": train_pred}).to_csv('train_predictions_etc_v1.csv',index=False)
print('Done.....')

('Training ', 1, 'of', 10.0)
('CV Score ', 0.45387359886779105)
('Training ', 2, 'of', 10.0)
('CV Score ', 0.45893583585076941)
('Training ', 3, 'of', 10.0)
('CV Score ', 0.45587385112755741)
('Training ', 4, 'of', 10.0)
('CV Score ', 0.45100526784396433)
('Training ', 5, 'of', 10.0)
('CV Score ', 0.45024065190654033)
('Training ', 6, 'of', 10.0)
('CV Score ', 0.45204595757345939)
('Training ', 7, 'of', 10.0)
('CV Score ', 0.46667250723527198)
('Training ', 8, 'of', 10.0)
('CV Score ', 0.44954018906884252)
('Training ', 9, 'of', 10.0)
('CV Score ', 0.44838269244805679)
('Training ', 10, 'of', 10.0)
('CV Score ', 0.45881458243129569)
Writing output files
Done.....


In [20]:
#Extra tree regressor
random.seed(123)
X_train = train
X_test = test
print('Training...')
extc = ExtraTreesClassifier(n_estimators=850,max_features= 60,criterion= 'entropy',min_samples_split= 4,
                            max_depth= 40, min_samples_leaf= 2, n_jobs = -1)      

extc.fit(X_train,target)

Training...
